## Install required libraries

- `azure-identity`: Azure Active Directory identity client library
- `azure.mgmt.confidentialledger`: Azure confidential ledger control plane client library
- `azure.confidentialledger`:  Azure confidential ledger data plane client library

In [ ]:
!pip install azure-identity azure.mgmt.confidentialledger azure.confidentialledger 

## Initialize

Import the required libraries.

In [ ]:
# Import the Azure authentication library

from azure.identity import DefaultAzureCredential, ManagedIdentityCredential, AzureCliCredential 

## Import the control plane sdk

from azure.mgmt.confidentialledger import ConfidentialLedger as ConfidentialLedgerAPI
from azure.mgmt.confidentialledger.models import ConfidentialLedger

# import the data plane sdk

from azure.confidentialledger import ConfidentialLedgerClient
from azure.confidentialledger.certificate import ConfidentialLedgerCertificateClient

We'll finish setup by setting some variables for use in your application: the resource group, the name of ledger you want to create, and two urls to be used by the data plane client library.

In [ ]:
resource_group = "azureailab01"
ledger_name = "azureailab01"
subscription_id = "82ed3e67-3850-491b-a098-cdca6aa71f3f"

identity_url = "https://identity.confidential-ledger.core.azure.com"
ledger_url = "https://" + ledger_name + ".confidential-ledger.azure.com"

## Use the control plane library

The control plane client library (azure.mgmt.confidentialledger) allows operations on ledgers, such as creation, modification, and deletion, listing the ledgers associated with a subscription, and getting the details of a specific ledger.

In our code, we will first create a control plane client by passing the ConfidentialLedgerAPI the credential variable and your Azure subscription ID (both of which are set above).

In [ ]:
confidential_ledger_mgmt = ConfidentialLedgerAPI(
    credential, subscription_id
)

We can now create a ledger using `begin_create`. The `begin_create` function requires three parameters: your resource group, a name for the ledger, and a "properties" object.

Create a properties dictionary with the following keys and values, and assign it to a variable.

In [ ]:
properties = {
    "location": "eastus",
    "tags": {},
    "properties": {
        "ledgerType": "Public",
        "aadBasedSecurityPrincipals": [],
    },
}

ledger_properties = ConfidentialLedger(**properties)

Now pass the resource group, the name of your ledger, and the properties object to `begin_create`.

In [ ]:
confidential_ledger_mgmt.ledger.begin_create(resource_group, ledger_name, ledger_properties)

To verify that your ledger was successfully created, view its details using the `get` function.

In [ ]:
myledger = confidential_ledger_mgmt.ledger.get(resource_group, ledger_name)

print("Here are the details of your newly created ledger:")
print (f"- Name: {myledger.name}")
print (f"- Location: {myledger.location}")
print (f"- ID: {myledger.id}")

Check some additional properties of the ledger.

In [ ]:
print(myledger.properties)

### Exercise:

Open the Azure portal, locate the Azure item of type `Confidential Ledger` and explore its properties.

## Use the data plane client library

Now that we have a ledger, we'll interact with it using the data plane client library (azure.confidentialledger).

First, we will generate and save a confidential ledger certificate.

In [ ]:
identity_client = ConfidentialLedgerCertificateClient(identity_url)
network_identity = identity_client.get_ledger_identity(
     ledger_id=ledger_name
)

ledger_tls_cert_file_name = "networkcert.pem"
with open(ledger_tls_cert_file_name, "w") as cert_file:
    cert_file.write(network_identity['ledgerTlsCertificate'])

Now we can use the network certificate, along with the ledger URL and our credentials, to create a confidential ledger client.

In [ ]:
# credential = ManagedIdentityCredential()
# credential = AzureCliCredential()
credential = DefaultAzureCredential()

In [ ]:
credential.get_token(ledger_url)

In [ ]:
ledger_client = ConfidentialLedgerClient(
     endpoint=ledger_url, 
     credential=credential,
     ledger_certificate_path=ledger_tls_cert_file_name
)

We are prepared to write to the ledger. We will do so using the `create_ledger_entry` function.

The print function will return the transaction ID of your write to the ledger, which can be used to retrieve the message you wrote to the ledger.

In [ ]:
sample_entry = {"contents": "Hellow immutable, blockchain-based world!"}
append_result = ledger_client.create_ledger_entry(entry=sample_entry)
print(append_result['transactionId'])